# 資料預處理

In [1]:
import os
os.chdir("C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Music\\Code")
import sys

from Get_file import Get_file   # return dirpath, dirnames, filenames
from FFT import FFT, PlotDataOnTimeDomain
from BandPassFilter import BandPassFilter, PlotDataOnFreqDomain, BandPassListFilter
import pyedflib
import numpy as np
import datetime
from Danny_ML_CLF import Danny_ML_CLF

import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA, PCA

%matplotlib inline

data_path_second = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Music\\第二首歌實驗數據"  # EEG Data direction

# Setting parameters
SampleFrequency = 128
DEBUG = True
LabelSize = 40
LowerFreqCut = 1
HigherFreqCut = 50
BandPassOrder = 9
channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
SAVE_FIG = False


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def CalculateZeros(NumpyArray):
    LogicWithZeros = (NumpyArray == 0)
    return np.sum(LogicWithZeros)
def Cal(minu, sec):
    mid = minu*60 + sec
    return [mid-0, mid+60]

# TimeSecondsi : Occur seconds init
# TimeSecondsf : Occur seconds final
def GrabMusic(SignalList, StudentName, TimeList, fs=128):
    channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
    Indexi = TimeList[0] * fs
    Indexf = TimeList[1] * fs
    
    DataNums = int(len(SignalList)/2)
    for i in range(DataNums):
        StdName = SignalList[2*i+1]
        if StdName ==  StudentName:
            LabelNum = SignalList[i*2].shape[0]
            ReturnAllChannelTimeSliceData = np.zeros([len(channel_we_use), (Indexf-Indexi)], dtype=np.float32)
            for ch in range(LabelNum):
                ReturnAllChannelTimeSliceData[ch, :] = SignalList[i*2][ch][Indexi:Indexf]
                
            if CalculateZeros(ReturnAllChannelTimeSliceData) == 0:
                print(StdName, ReturnAllChannelTimeSliceData.shape)
                return ReturnAllChannelTimeSliceData
            else:
                print("Warning: Have Zero Values in the Numpy array !!!")
    print('None')
    return None

In [3]:
# 抓取第三首歌的數據
os.chdir(data_path_second)
PLOT_FLAG = False
Files = []
RawSignal = []
for _,files,_ in os.walk(data_path_second):
    if files != []:
        Files.append(files)
PeopleFiles = Files[0]  # Get All People files in this list

for name in PeopleFiles:
    _,_,file_names = Get_file(data_path_second+'\\'+name+'\\'+'原始資料')
    print(name)
    for fn in file_names:  # fn os file name
        if 'expx.edf' in fn:
            StudentID = fn.split('.')[0][0:7]
            data_dir = data_path_second+'\\'+ name + '\\' + '原始資料' + '\\' + fn
            os.chdir(data_path_second+'\\'+ name + '\\' + '原始資料' + '\\')
            fedf = pyedflib.EdfReader(fn)
            n = fedf.signals_in_file
            signal_labels = np.asarray(fedf.getSignalLabels())
            sigbufs = np.zeros((len(channel_we_use), fedf.getNSamples()[0]))
            print('Data Shape : ', sigbufs.shape)
            CHCNT = 0
            for i in range(n):
                if i in channel_we_use:
                    sigbufs[CHCNT,:] = fedf.readSignal(i)
                    CHCNT+=1
                    if PLOT_FLAG:
                        plt.figure(figsize=(12,8))
                        x = np.linspace(0, sigbufs[i,:].shape[0]/128, sigbufs[i,:].shape[0])
                        plt.plot(x, sigbufs[i,:], 'r*')
                        plt.show()
            if CalculateZeros(sigbufs):
                print('Warning Have All zero in array !!!')
            RawSignal.append(sigbufs)
            RawSignal.append(name)

print('Data Length : ', len(RawSignal))
print('奇數是腦波數據\n偶數是實驗者')

于綺
Data Shape :  (14, 188672)
俊佑
Data Shape :  (14, 151808)
博翔
Data Shape :  (14, 188800)
吉淳(D)
Data Shape :  (14, 188672)
名璿
Data Shape :  (14, 76544)
品丞
Data Shape :  (14, 76416)
士瑾(C)
Data Shape :  (14, 188672)
大哲
Data Shape :  (14, 189056)
子禎
Data Shape :  (14, 192000)
宗穎
Data Shape :  (14, 188672)
庭瑄
Data Shape :  (14, 188800)
弼仲(A)
Data Shape :  (14, 188672)
德全
Data Shape :  (14, 188416)
思樺(D)
Data Shape :  (14, 188288)
思毅
Data Shape :  (14, 188672)
憲威(D)
Data Shape :  (14, 188672)
承翰
Data Shape :  (14, 188672)
易達
Data Shape :  (14, 164992)
昕凱
Data Shape :  (14, 188672)
智宇
Data Shape :  (14, 187776)
書暐
Data Shape :  (14, 188672)
柏儒
Data Shape :  (14, 188672)
柏凱(D)
Data Shape :  (14, 188800)
楊鵬
Data Shape :  (14, 188672)
浩恩
Data Shape :  (14, 188672)
育銘(D)
Data Shape :  (14, 187008)
鉦天(C)
Data Shape :  (14, 188544)
韋蒼
Data Shape :  (14, 188672)
韡潪(C)
Data Shape :  (14, 188800)
駿傑
Data Shape :  (14, 188160)
Data Length :  60
奇數是腦波數據
偶數是實驗者


In [27]:
# 沒有雙腦波差的數據
NormalSignal = []
NormalSignal.append(GrabMusic(RawSignal, '于綺', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '于綺', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '于綺', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '于綺', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '于綺', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '大哲', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '大哲', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '大哲', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '大哲', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '大哲', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '名璿', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '名璿', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '名璿', Cal(4,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '宗穎', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '宗穎', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '宗穎', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '宗穎', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '宗穎', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '承翰', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '承翰', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '承翰', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '承翰', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '承翰', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '易達', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '易達', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '易達', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '易達', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '易達', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '思毅', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思毅', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思毅', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '思樺(D)', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思樺(D)', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思樺(D)', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思樺(D)', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '思樺(D)', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '柏儒', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏儒', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏儒', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏儒', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '柏儒', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '韋蒼', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '韋蒼', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '韋蒼', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '韋蒼', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '韋蒼', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '書暐', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '書暐', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '書暐', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '書暐', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '書暐', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '浩恩', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '浩恩', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '浩恩', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '浩恩', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '浩恩', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '博翔', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '博翔', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '博翔', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '博翔', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '博翔', Cal(6,0), fs=128))

NormalSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(2,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(3,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(4,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(5,0), fs=128))
NormalSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(6,0), fs=128))

print('Normal Length : ', len(NormalSignal))

于綺 (14, 7680)
于綺 (14, 7680)
于綺 (14, 7680)
于綺 (14, 7680)
于綺 (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
大哲 (14, 7680)
大哲 (14, 7680)
大哲 (14, 7680)
大哲 (14, 7680)
大哲 (14, 7680)
名璿 (14, 7680)
名璿 (14, 7680)
名璿 (14, 7680)
育銘(D) (14, 7680)
育銘(D) (14, 7680)
育銘(D) (14, 7680)
育銘(D) (14, 7680)
育銘(D) (14, 7680)
宗穎 (14, 7680)
宗穎 (14, 7680)
宗穎 (14, 7680)
宗穎 (14, 7680)
宗穎 (14, 7680)
承翰 (14, 7680)
承翰 (14, 7680)
承翰 (14, 7680)
承翰 (14, 7680)
承翰 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
思毅 (14, 7680)
思毅 (14, 7680)
思毅 (14, 7680)
思樺(D) (14, 7680)
思樺(D) (14, 7680)
思樺(D) (14, 7680)
思樺(D) (14, 7680)
思樺(D) (14, 7680)
柏凱(D) (14, 7680)
柏凱(D) (14, 7680)
柏凱(D) (14, 7680)
柏凱(D) (14, 7680)
柏凱(D) (14, 7680)
柏儒 (14, 7680)
柏儒 (14, 7680)
柏儒 (14, 7680)
柏儒 (14, 7680)
柏儒 (14, 7680)
韋蒼 (14, 7680)
韋蒼 (14, 7680)
韋蒼 (14, 7680)
韋蒼 (14, 7680)
韋蒼 (14, 7680)
書暐 (14, 7680)
書暐 (14, 7680)
書暐 (14, 7680)
書暐 (14, 7680)
書暐 (14, 7680)
浩恩 (14, 7680)
浩恩

In [14]:
# 有雙腦波差的數據
GrabSignal = []
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(10,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(12,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(14,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(16,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(17,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(18,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '士瑾(C)', Cal(22,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '大哲', Cal(12,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '子禎', Cal(20,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(6,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(7,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(8,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(9,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(10,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(11,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(12,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(13,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(14,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(15,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(16,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(17,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(18,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(19,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(20,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(21,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(22,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(23,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(24,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '名璿', Cal(25,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '育銘(D)', Cal(22,30), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(9,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(10,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(13,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(15,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(16,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(18,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '易達', Cal(20,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '昕凱', Cal(4,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '昕凱', Cal(18,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(1,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(2,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(3,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(4,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(5,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(6,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(7,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(8,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(9,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(10,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(11,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(12,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(13,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(14,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(15,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(16,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(17,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(18,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(19,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(20,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(21,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(22,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(23,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '俊佑', Cal(24,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '思毅', Cal(2,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '思毅', Cal(14,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '柏凱(D)', Cal(18,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '庭瑄', Cal(6,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(9,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(12,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(17,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(20,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(21,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '弼仲(A)', Cal(23,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(1,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(2,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(3,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(4,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(5,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(6,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(7,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(8,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(9,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(10,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(11,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(12,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(13,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(14,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(15,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(16,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(17,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(18,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(19,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(20,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(21,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(22,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(23,0), fs=128))
# GrabSignal.append(GrabMusic(RawSignal, '智宇', Cal(24,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '鉦天(C)', Cal(1,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '鉦天(C)', Cal(2,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '憲威(D)', Cal(20,0), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '駿傑', Cal(15,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '駿傑', Cal(23,30), fs=128))

GrabSignal.append(GrabMusic(RawSignal, '韡潪(C)', Cal(14,0), fs=128))
GrabSignal.append(GrabMusic(RawSignal, '韡潪(C)', Cal(23,30), fs=128))

print('Grab Length : ', len(GrabSignal))

士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
士瑾(C) (14, 7680)
大哲 (14, 7680)
子禎 (14, 7680)
名璿 (14, 7680)
名璿 (14, 7680)
育銘(D) (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
易達 (14, 7680)
昕凱 (14, 7680)
昕凱 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
俊佑 (14, 7680)
思毅 (14, 7680)
思毅 (14, 7680)
柏凱(D) (14, 7680)
庭瑄 (14, 7680)
弼仲(A) (14, 7680)
弼仲(A) (14, 7680)
弼仲(A) (14, 7680)
弼仲(A) (14, 7680)
弼仲(A) (14, 7680)
弼仲(A) (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 (14, 7680)
智宇 

# 1分鐘的數據全部儲存下來

In [61]:
PLOT_FLAG = False
# 4~8 Hz as my features (5 dimensions)

signal_features = np.zeros([4740, 25], dtype=np.float32)   # 79 * 60
cnt = 0
for i in range(len(GrabSignal)):
    Split_Times = int(GrabSignal[i].shape[1]/128)
    Split_Num = 0
    for sp in range(Split_Times):
        buff = np.zeros([len(channel_we_use), 25], dtype=np.float32)
        for ch in range(len(channel_we_use)):
            data = GrabSignal[i][ch][Split_Num:Split_Num+SampleFrequency]
            BandPassData = BandPassFilter(data, LowerFreqCut, HigherFreqCut, SampleFrequency)
            SignalFreqX, FFTData = FFT(BandPassData, SampleFrequency, SampleFrequency)
            buff[ch,:] = FFTData[3:28]  # 4~28 Hz
            if PLOT_FLAG:
                plt.figure(figsize=(20,8))
                plt.plot(SignalFreqX[3:28], FFTData[3:28], '*-')
                plt.show()
        if CalculateZeros(buff) != 0:
            print('Warning !!!')
        Collect_Signal = np.mean(buff, axis=0)
        signal_features[cnt, :] = Collect_Signal
        cnt+=1
        Split_Num = Split_Num + SampleFrequency
if CalculateZeros(signal_features) != 0:
    print('Warning !!!')
print('Sleep Data Shape : ', signal_features.shape)   

Sleep Data Shape :  (4740, 25)


In [62]:
PLOT_FLAG = False
# 4~28 Hz as my features (5 dimensions)
# Normal

signal_Normal = np.zeros([4860, 25], dtype=np.float32)  # 65 * 60
cnt = 0
for i in range(len(NormalSignal)):
    Split_Times = int(NormalSignal[i].shape[1]/128)
    Split_Num = 0
    for sp in range(Split_Times):
        buff = np.zeros([len(channel_we_use), 25], dtype=np.float32)
        for ch in range(len(channel_we_use)):
            data = NormalSignal[i][ch][Split_Num:Split_Num+SampleFrequency]
            BandPassData = BandPassFilter(data, LowerFreqCut, HigherFreqCut, SampleFrequency)
            SignalFreqX, FFTData = FFT(BandPassData, SampleFrequency, SampleFrequency)
            buff[ch,:] = FFTData[3:28]  # 4~28 Hz
            if PLOT_FLAG:
                plt.figure(figsize=(20,28))
                plt.plot(SignalFreqX[3:28], FFTData[3:28], '*-')
                plt.show()
        if CalculateZeros(buff) != 0:
            print('Warning !!!')
        Collect_Signal = np.mean(buff, axis=0)
        signal_Normal[cnt, :] = Collect_Signal
        cnt+=1
        Split_Num = Split_Num + SampleFrequency
if CalculateZeros(signal_Normal) != 0:
    print('Warning !!!')
print('Normal Data Shape : ', signal_Normal.shape)   

Normal Data Shape :  (4860, 25)


# 一分鐘的數據，當成一個

In [100]:
PLOT_FLAG = False
# 4~8 Hz as my features (5 dimensions)

signal_features = np.zeros([79, 25], dtype=np.float32)   # 79
cnt = 0
for i in range(len(GrabSignal)):
    buff = np.zeros([len(channel_we_use), 25], dtype=np.float32)
    for ch in range(len(channel_we_use)):
        data = GrabSignal[i][ch][:]
        BandPassData = BandPassFilter(data, LowerFreqCut, HigherFreqCut, SampleFrequency)
        SignalFreqX, FFTData = FFT(BandPassData, SampleFrequency, SampleFrequency)
        buff[ch,:] = FFTData[3:28]  # 4~28 Hz
        if PLOT_FLAG:
            plt.figure(figsize=(20,8))
            plt.plot(SignalFreqX[3:28], FFTData[3:28], '*-')
            plt.show()
    if CalculateZeros(buff) != 0:
        print('Warning !!!')
    Collect_Signal = np.mean(buff, axis=0)
    signal_features[cnt, :] = Collect_Signal
    cnt+=1

if CalculateZeros(signal_features) != 0:
    print('Warning !!!')
print('Sleep Data Shape : ', signal_features.shape)   

Sleep Data Shape :  (79, 25)


In [102]:
PLOT_FLAG = False
# 4~28 Hz as my features (5 dimensions)
# Normal

signal_Normal = np.zeros([81, 25], dtype=np.float32)  # 81
cnt = 0
for i in range(len(NormalSignal)):
    buff = np.zeros([len(channel_we_use), 25], dtype=np.float32)
    for ch in range(len(channel_we_use)):
        data = NormalSignal[i][ch][:]
        BandPassData = BandPassFilter(data, LowerFreqCut, HigherFreqCut, SampleFrequency)
        SignalFreqX, FFTData = FFT(BandPassData, SampleFrequency, SampleFrequency)
        buff[ch,:] = FFTData[3:28]  # 4~28 Hz
        if PLOT_FLAG:
            plt.figure(figsize=(20,28))
            plt.plot(SignalFreqX[3:28], FFTData[3:28], '*-')
            plt.show()
    if CalculateZeros(buff) != 0:
        print('Warning !!!')
    Collect_Signal = np.mean(buff, axis=0)
    signal_Normal[cnt, :] = Collect_Signal
    cnt+=1
if CalculateZeros(signal_Normal) != 0:
    print('Warning !!!')
print('Normal Data Shape : ', signal_Normal.shape)  

Normal Data Shape :  (81, 25)


In [103]:
# Create Labels
label1 = np.ones(signal_features.shape[0]) * 1
label2 = np.ones(signal_Normal.shape[0]) * 0
label = np.hstack((label1, label2))

data = np.vstack((signal_features, signal_Normal))

print(label.shape)
print(data.shape)

(160,)
(160, 25)


In [134]:
clf = Danny_ML_CLF()
train_X, test_X, train_y, test_y = clf.Split_data(data, label, test_size=0.3, Standard=True)
clf.Fit_value(train_X, train_y)
clf.Train()
clf.Report(train_X, train_y, [0,1], show_cm=False)
clf.Report2txt('Train.txt')
print(clf.Score())
print()
clf.Report(test_X, test_y, [0,1], show_cm=False)
clf.Report2txt('Test.txt')
print(clf.Score())
print('Report Okay !\n\n')

C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


SVM Score :  0.6607142857142857
Tree Score :  0.9642857142857143
Bayes Score :  0.5178571428571429
KNN Score :  1.0
XGB Score :  0.9910714285714286
Stacking Score :  1.0
Voting Score :  0.9910714285714286
Bagging Score :  1.0
RF Score :  1.0
Adaboost Score :  1.0
None

SVM Score :  0.4375
Tree Score :  0.5416666666666666
Bayes Score :  0.625
KNN Score :  0.5833333333333334
XGB Score :  0.5833333333333334
Stacking Score :  0.5833333333333334
Voting Score :  0.6666666666666666
Bagging Score :  0.625
RF Score :  0.625
Adaboost Score :  0.5625
None
Report Okay !




C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [135]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
model = Sequential()
model.add(Dense(500, input_dim=25))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [136]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_X, train_y, epochs=50, batch_size=10)

Epoch 1/50
112/112 [==============================] - 2s 18ms/step - loss: 0.7225 - acc: 0.5000
Epoch 2/50
112/112 [==============================] - 0s 761us/step - loss: 0.7025 - acc: 0.5804
Epoch 3/50
112/112 [==============================] - 0s 770us/step - loss: 0.6713 - acc: 0.6518
Epoch 4/50
112/112 [==============================] - 0s 725us/step - loss: 0.6884 - acc: 0.5625
Epoch 5/50
112/112 [==============================] - 0s 689us/step - loss: 0.5623 - acc: 0.7321
Epoch 6/50
112/112 [==============================] - 0s 725us/step - loss: 0.5998 - acc: 0.7321
Epoch 7/50
112/112 [==============================] - 0s 734us/step - loss: 0.5732 - acc: 0.6964
Epoch 8/50
112/112 [==============================] - 0s 761us/step - loss: 0.5491 - acc: 0.6964
Epoch 9/50
112/112 [==============================] - 0s 725us/step - loss: 0.5303 - acc: 0.7054
Epoch 10/50
112/112 [==============================] - 0s 716us/step - loss: 0.5323 - acc: 0.7589
Epoch 11/50
112/112 [=========

In [137]:
score = model.evaluate(test_X, test_y, batch_size=200)
score

48/48 [==============================] - 1s 12ms/step


[0.6303123831748962, 0.6041666865348816]

In [138]:
model.predict(test_X)

array([[0.18827231],
       [0.49170136],
       [0.6765259 ],
       [0.77410907],
       [0.4434272 ],
       [0.6266734 ],
       [0.6158044 ],
       [0.7193323 ],
       [0.29838625],
       [0.85087144],
       [0.36580446],
       [0.35884503],
       [0.6866505 ],
       [0.6551612 ],
       [0.01903065],
       [0.44161874],
       [0.8760926 ],
       [0.34541833],
       [0.6951799 ],
       [0.72928405],
       [0.5577903 ],
       [0.47726163],
       [0.44956732],
       [0.654976  ],
       [0.6975136 ],
       [0.60779643],
       [0.6331484 ],
       [0.65907604],
       [0.5565915 ],
       [0.0736778 ],
       [0.26545575],
       [0.6283327 ],
       [0.7316916 ],
       [0.09254036],
       [0.07427698],
       [0.6424223 ],
       [0.76841384],
       [0.47612283],
       [0.4007084 ],
       [0.3819894 ],
       [0.19226712],
       [0.5903374 ],
       [0.46096015],
       [0.76637167],
       [0.5275283 ],
       [0.7604239 ],
       [0.5603532 ],
       [0.450

In [139]:
test_y

array([1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.])